In [1]:
import numpy as np
import pandas as pd

In [62]:
feature_path ='./feature/'
res_path = './res/'
data_path = './data/'
second_round_path = './second round/'

In [63]:
transaction_df = pd.read_csv(data_path+'transaction_train_new.csv')
operation_df =  pd.read_csv(data_path+'operation_train_new.csv')
label = pd.read_csv(data_path+'tag_train_new.csv')

transaction_test = pd.read_csv(data_path+'transaction_round1_new.csv')
operation_test = pd.read_csv(data_path+'operation_round1_new.csv')

# transaction_test = pd.read_csv(second_round_path+'test_transaction_round2.csv')
# operation_test = pd.read_csv(second_round_path+'test_operation_round2.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [386]:
def merge_count(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].count().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_nunique(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].nunique().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_value_count(df1,df2,col, name=None, divisor=None):
    if name is None:
        name = col[1]
    tmp = df1.groupby(col).size().reset_index().rename(columns = {0:'cnt'})
    df = tmp.pivot(index=col[0],columns=col[1],values='cnt').reset_index()
    if divisor is not None:
        df = df.apply(lambda x: [x.iloc[0], *(x.iloc[1:] / divisor[x.iloc[0]])], axis=1, result_type='broadcast')
    cname = [col[0]]
    for index in range(1,len(df.columns)):
        cname.append(str(name)+'_'+str(df.columns[index]))
    df.columns=cname
    df = df.fillna(0)
    df2 = df2.merge(df,on=str(col[0]),how='left')
    del df,df1
    return df2

In [387]:
import geohash
def get_op_fea(operation_df):
    op_counts = operation_df.groupby('UID').size()
    
    #op_day
    op_fea = operation_df[['UID']].drop_duplicates()
    tmp = operation_df.groupby('UID')['day'].agg([max,min,np.mean]).reset_index()
    tmp.columns=['UID','op_day_max','op_day_min','op_day_mean']
    op_fea = pd.merge(op_fea,tmp,on='UID',how='left')
    
    #Geo info
    geo = operation_df['geo_code'].apply(lambda x: ((x, x) if pd.isna(x) else geohash.decode(x)))
    operation_df['latitude'] = geo.apply(lambda x: x[0])
    operation_df['longtitude'] = geo.apply(lambda x: x[1])
    tmp = operation_df.groupby('UID')['latitude'].agg([max,min,np.mean]).reset_index()
    tmp.columns = ['UID', 'op_latitude_max', 'op_latitude_min', 'op_latitude_mean']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    tmp = operation_df.groupby('UID')['longtitude'].agg([max,min,np.mean]).reset_index()
    tmp.columns = ['UID', 'op_longtitude_max', 'op_longtitude_min', 'op_longtitude_mean']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    #op_mode count
    op_fea = merge_count(operation_df,op_fea,'UID','mode','op_cnt')
    op_fea = merge_nunique(operation_df,op_fea,'UID','mode','op_mode_nunique')
    #success count
    op_fea = merge_count(operation_df[operation_df.success==0],op_fea,'UID','mode','op_fail_cnt')
    op_fea = merge_count(operation_df[operation_df.success==1],op_fea,'UID','mode','op_success_cnt')
    op_fea['op_fail_cnt'] = op_fea['op_fail_cnt'].fillna(0)
    op_fea['op_success_cnt'] = op_fea['op_success_cnt'].fillna(0)
    
    op_fea = merge_value_count(operation_df,op_fea,['UID','day'],'op_days', divisor=op_counts)
    #op_time
    operation_df['op_hour'] = operation_df['time'].apply(lambda x:int(x.split(':')[0]))
    tmp = operation_df.groupby('UID')['op_hour'].agg([max,min,np.mean]).reset_index()
    tmp.columns=['UID','op_hour_max','op_hour_min','op_hour_mean']
    op_fea = pd.merge(op_fea,tmp,on='UID',how='left')
    op_fea = merge_value_count(operation_df,op_fea,['UID','op_hour'], divisor=op_counts)
    
    #op_os
    for col in ['os','version','device1','device2','device_code1','device_code2','mac1','ip1','ip2','device_code3','mac2','wifi','ip1_sub','ip2_sub']:
        op_fea = merge_nunique(operation_df,op_fea,'UID',col,'op_'+col+'_nunique')
    return op_fea

def get_trans_fea(transaction_df):
    trans_counts = transaction_df.groupby('UID').size()
    
    trans_fea = transaction_df[['UID']].drop_duplicates()
    #trans_channel
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','channel'], 'trans_channel', divisor=trans_counts)
    trans_fea = merge_count(transaction_df,trans_fea,'UID','channel','trans_cnt')
    trans_fea = merge_nunique(transaction_df,trans_fea,'UID','channel','trans_channel_nunique')
    
    # Days
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','day'],'trans_day', divisor=trans_counts)
    
    geo = transaction_df['geo_code'].apply(lambda x: ((x, x) if pd.isna(x) else geohash.decode(x)))
    transaction_df['latitude'] = geo.apply(lambda x: x[0])
    transaction_df['longtitude'] = geo.apply(lambda x: x[1])
    tmp = transaction_df.groupby('UID')['latitude'].agg([max,min,np.mean]).reset_index()
    tmp.columns = ['UID', 'trans_latitude_max', 'trans_latitude_min', 'trans_latitude_mean']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')
    tmp = transaction_df.groupby('UID')['longtitude'].agg([max,min,np.mean]).reset_index()
    tmp.columns = ['UID', 'trans_longtitude_max', 'trans_longtitude_min', 'trans_longtitude_mean']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')
    
    #trans time
    transaction_df['trans_hour'] = transaction_df['time'].apply(lambda x:int(x.split(':')[0]))
    tmp = transaction_df.groupby('UID')['trans_hour'].agg([max,min,np.mean]).reset_index()
    tmp.columns=['UID','trans_hour_max','trans_hour_min','trans_hour_mean']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','trans_hour'], divisor=trans_counts)

    for col in ['trans_type2','market_type']:
        trans_fea = merge_value_count(transaction_df,trans_fea,['UID',col], 'trans_%s'%col, divisor=trans_counts)
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
    for col in ['trans_type1','merchant','code1','code2','acc_id1','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','acc_id2','acc_id3','market_code','ip1_sub']:
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
    return trans_fea


In [388]:
operation_features = get_op_fea(operation_df)
transaction_features = get_trans_fea(transaction_df)

In [389]:
operation_test_feat = get_op_fea(operation_test)
transaction_test_feat = get_trans_fea(transaction_test)

In [404]:
train_features = operation_features.merge(transaction_features, on='UID', how='outer')
train_data = train_features.merge(label,on='UID',how='left')
train_data = train_data.drop('UID', axis=1)
test_features = operation_test_feat.merge(transaction_test_feat, on='UID', how='outer').sort_values(['UID'])
UIDs = test_features['UID']
test_features = test_features.drop('UID', axis=1)

columns_to_drop = [col for col in train_data.columns if col not in test_features.columns and col != 'Tag']
train_data = train_data.drop(columns_to_drop, axis=1)
columns_to_drop = [col for col in test_features.columns if col not in train_data.columns and col != 'Tag']
test_features = test_features.drop(columns_to_drop, axis=1)

In [405]:
import sklearn.preprocessing
def preprocess(x):
    scaler = sklearn.preprocessing.MinMaxScaler()
    return scaler.fit_transform(x)

In [406]:
def tpr_scorer(estimator, x, y_true):
    y_predict = estimator.predict_proba(x)[:, 1]
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [407]:
import sklearn.model_selection
import sklearn.metrics
import lightgbm as lgb
# import xgboost as xgb
def cv(x, y, params={}, splits=5):
    clf = lgb.LGBMClassifier(**params)
#     clf = xgb.XGBClassifier(**params)
    kfold = sklearn.model_selection.StratifiedKFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'tpr': tpr_scorer,
        'accuracy': 'accuracy',
        'f1': 'f1_micro',
        'roc_auc': 'roc_auc'
    })
    return cv_score

In [420]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#     'objective': 'binary:logistic',
    'max_depth': 7,
    'num_leaves': 80,
#     'max_leaf_nodes': 50,
    'min_child_samples': 400,
    'n_estimators': 3000,
    'learning_rate': 0.05,
    'scale_pos_weight': float(label[label.Tag==0].shape[0]) / label[label.Tag==1].shape[0],
    'boost_from_average': True,
    'min_child_weight': 4,
#     'gamma': 3,
    'reg_alpha': 3,
    'reg_lambda': 10,
    'subsample': 0.7,
    'colsample_bytree':0.7, 
    'subsample_freq': 1,
    'n_jobs': -1,
#     'booster':'gbtree',
}
cv_result = cv(train_data.drop('Tag', axis=1).values, train_data.loc[:, 'Tag'].values, params=params, splits=5)
for scorer, score in cv_result.items():
    print('%s: %s' % (scorer, score))
    print('Average %s: %f' % (scorer, score.mean()))

fit_time: [14.25155473 12.84871292 22.5232923  14.49269104 13.6809299 ]
Average fit_time: 15.559436
score_time: [0.97285914 0.78975105 0.97788382 0.82235909 0.86986303]
Average score_time: 0.886543
test_tpr: [0.77187865 0.78039673 0.75204201 0.71586931 0.75087515]
Average test_tpr: 0.754212
train_tpr: [1. 1. 1. 1. 1.]
Average train_tpr: 1.000000
test_accuracy: [0.96215523 0.95894804 0.96199487 0.9555805  0.9584603 ]
Average test_accuracy: 0.959428
train_accuracy: [0.99887744 0.99867698 0.99891753 0.99919817 0.99895767]
Average train_accuracy: 0.998926
test_f1: [0.96215523 0.95894804 0.96199487 0.9555805  0.9584603 ]
Average test_f1: 0.959428
train_f1: [0.99887744 0.99867698 0.99891753 0.99919817 0.99895767]
Average train_f1: 0.998926
test_roc_auc: [0.96830212 0.96713178 0.96917504 0.95625431 0.96585933]
Average test_roc_auc: 0.965345
train_roc_auc: [0.99999346 0.99999231 0.99998327 0.99998361 0.99998187]
Average train_roc_auc: 0.999987


In [410]:
def train(x, y, params={}):
    clf = lgb.LGBMClassifier(**params)
#     clf = xgb.XGBClassifier(**params)
    clf.fit(x, y)
    return clf

In [411]:
model = train(train_data.drop('Tag', axis=1).values, train_data.loc[:, 'Tag'].values, params=params)

In [412]:
result = model.predict_proba(test_features.values)[:, 1]

In [413]:
result_frame = pd.DataFrame()
result_frame['UID'] = UIDs
result_frame['Tag'] = result

In [414]:
result_frame.to_csv(res_path + 'res.csv', index=False)

In [415]:
(result > 0.5).sum() / len(result)

0.08692864927238926